In [1]:
!pip install Bio

import requests
from io import StringIO
from Bio import SeqIO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.4 MB/s eta 0:00:00


In [2]:
def read_from_drive(url):
    
    file_id = url.split('/')[-1]
    download_url='https://drive.google.com/uc?export=download&id=' + file_id
    
    url_text = requests.get(download_url).text
    raw_file = StringIO(url_text)
    
    return raw_file

In [3]:
pealeii_url = "https://drive.google.com/file/d/1omKelb4BXrFlFj2a3Pjs79lIAa2hwQVU"
pealeii_fasta = read_from_drive(pealeii_url)

gfl_url = "https://drive.google.com/file/d/1orrUCOB6Ji2cZD7B05PLuOdKEeyjuPK5"
gfl_bam = read_from_drive(gfl_url)

gill_url = "https://drive.google.com/file/d/1p4FwGoHTMnAenSX85b9rybc0OSY8jwUB"
gill_bam = read_from_drive(gill_url)

In [4]:
working_dir = "/kaggle/working/"

In [5]:
ids = []
sequences = []
orf_starts = []
orf_ends = []

with open(working_dir + "outfile.txt", "w") as out:

    for seq in SeqIO.parse(pealeii_fasta, "fasta"):
    
        out.write(seq.id + "\n")
    
        ids.append(seq.id)
        sequences.append(seq.seq)
    
        desc = seq.description.split("\t")
        orf_starts.append(desc[2])
        orf_ends.append(desc[4])